In [1]:
import sys
import os

# Add the path to `database_scripts` folder
sys.path.append(os.path.abspath("../database_scripts"))
import  db_utils as du
import create_tables as ct


conn = ct.create_or_open_database()
df_fahrrad_Zähler = du.fetch_data_df('Standorten_Zählstelle',conn)
df_mess_Fahrrad= du.fetch_data_df('Messdaten_Fahrrad',conn)
df_bezirke= du.fetch_data_df('Bezirke',conn)
df_date = du.fetch_data_df('Date_dim',conn)
df_time = du.fetch_data_df('Time_dim',conn)
df_wetter = du.fetch_data_df('Wetter',conn)
#type conversion inside the dataframe
df_time['TimeID'] = df_time['TimeID'].astype('int')
df_date['DateID'] = df_date['DateID'].astype('int')
df_wetter['TimeID'] = df_wetter['TimeID'].astype('int')
df_wetter['DateID'] = df_wetter['DateID'].astype('int32')
df_date['year'] = df_date['year'].astype('int')
df_date['month'] = df_date['month'].astype('int')
df_date['day'] = df_date['day'].astype('int')
df_date['is_holiday'] = df_date['is_holiday'].astype('bool')
df_date['quarter'] = df_date['quarter'].astype('int')
df_mess_Fahrrad['TimeID'] = df_mess_Fahrrad['TimeID'].astype('int')
df_mess_Fahrrad['DateID'] = df_mess_Fahrrad['DateID'].astype('int')
#df_mess_Fahrrad['Wert'] = df_mess_Fahrrad['Wert'].astype('int32')

df_wetter['temperature_2m (°C)'] = df_wetter['temperature_2m (°C)'].astype('float32')
df_wetter['relative_humidity_2m (%)'] = df_wetter['relative_humidity_2m (%)'].astype('float32')
df_wetter['rain (mm)'] = df_wetter['rain (mm)'].astype('float32')
df_wetter['snowfall (cm)'] = df_wetter['snowfall (cm)'].astype('float32')
df_wetter['cloud_cover (%)'] = df_wetter['cloud_cover (%)'].astype('float32')
#Feature Engineering:
#add a new colum for Zählstelle to be converted to number
df_fahrrad_Zähler['Zählstelle_ID'] = range(0, df_fahrrad_Zähler.shape[0])
df_fahrrad_Zähler['Zählstelle_ID'] = df_fahrrad_Zähler['Zählstelle_ID'].astype('int')
df_bezirke['Bezirk_ID'] =  range(0, df_bezirke.shape[0])
df_bezirke['Bezirk_ID'] = df_bezirke['Bezirk_ID'].astype('int')

def time_of_the_day_as_number(time_of_the_day):
    if time_of_the_day == 'nachts': 
        return 0
    if time_of_the_day == 'morgens':
        return 1
    if time_of_the_day == 'vormittags':
        return 2
    if time_of_the_day == 'mittags':
        return 3
    if time_of_the_day == 'nachmittags':
        return 4
    else:
        return 5
df_time['timeoftheday']=df_time['time_of_the_day'].apply(time_of_the_day_as_number)
df_time['timeoftheday'] = df_time['timeoftheday'].astype('int')
df_merged = df_bezirke.merge(df_fahrrad_Zähler, on='Bezirk', how='left')
df_merged = df_merged.merge(df_mess_Fahrrad, on='Zählstelle', how='left')
df_merged = df_merged.merge(df_wetter, on=['DateID','TimeID', 'Bezirk'], how='left')
df_merged = df_merged.merge(df_date, on='DateID', how='left')
df_merged = df_merged.merge(df_time, on='TimeID', how='left')
#print(df_merged.info())
df_merged.drop(['time_of_the_day','Breitengrad', 'Längengrad', 'Geometry','Bezirk','Date','Beschreibung','Installationsdatum','Zählstelle'], axis = 1, inplace= True)

Establish SQLite connection.


In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC  # For classification; use SVR for regression
from sklearn import  metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np

In [3]:
#Normalisierung
scalar = StandardScaler()
X = df_merged.drop(['Wert'], axis = 1)
y = df_merged['Wert']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)
X_test_scaled = scalar.fit_transform(X_test)
X_train_scaled = scalar.fit_transform(X_train)


In [4]:
#püfen RandomForest
rf_model = RandomForestRegressor(n_estimators=100, random_state= 42)
rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

KeyboardInterrupt: 

In [ ]:
#evaluate ranomForest
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
#prüfen SupportVector SVC
# Step 2: Set up hyperparameter grid
param_grid = {
    'kernel': ['poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'degree': [3, 5],  # For polynomial kernel
}

# Step 3: Perform grid search with cross-validation
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Step 4: Best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Step 5: Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate performance

accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy}")


In [ ]:
#prüfen KNN
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=5)  # You can tune k as a hyperparameter
knn_model.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = knn_model.predict(X_test)

# Step 5: Evaluate the model
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')